# Day 20
# Residual Connections

- Goal:
    1. Understand how residual blocks work.
    2. Build the key parts of Inception-ResNet.

## (a) Residual Block V1, V2

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import Concatenate, Lambda
from tensorflow.keras import backend as K

def ResidualV1_block(input_tensor, kernel_size, filters, stage, block):
    
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    
    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    
    return x

def ResidualV2_block(input_tensor, kernel_size, filters, stage, block):
    
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(x)
    
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters3, kernel_size, padding='same', name=conv_name_base + '2c')(x)
    
    x = layers.add([x, input_tensor])
    
    return x

## (b) Residual Inception Modules

In [2]:
# from Day19
def Conv2d_bn(x, filters, kernel_size, padding='same', strides=(1, 1), normalizer=True, activation='relu', name=None):
    
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    
    # determine the axis that should be normalized (typically the features axis)
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding, use_bias=False, name=conv_name)(x)
    
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    
    return x

In [3]:
def InceptionResNet_block(x, scale, block_type, activation='relu'):    # scale: scaling factor of the output of the inception module
    
    if block_type == 'IncRes_A':
        branch_0 = Conv2d_bn(x, 32, 1)
        branch_1 = Conv2d_bn(x, 32, 1)
        branch_1 = Conv2d_bn(branch_1, 32, 3)
        branch_2 = Conv2d_bn(x, 32, 1)
        branch_2 = Conv2d_bn(branch_2, 48, 3)
        branch_2 = Conv2d_bn(branch_2, 64, 3)
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'IncRes_B':
        branch_0 = Conv2d_bn(x, 192, 1)
        branch_1 = Conv2d_bn(x, 128, 1)
        branch_1 = Conv2d_bn(branch_1, 160, [1, 7])
        branch_1 = Conv2d_bn(branch_1, 192, [7, 1])
        branches = [branch_0, branch_1]
    elif block_type == 'IncRes_C':
        branch_0 = Conv2d_bn(x, 192, 1)
        branch_1 = Conv2d_bn(x, 192, 1)
        branch_1 = Conv2d_bn(branch_1, 224, [1, 3])
        branch_1 = Conv2d_bn(branch_1, 256, [3, 1])
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "block35", "block17" or "block8", '
                         'but got: ' + str(block_type))
    mixed = Concatenate(axis=3)(branches)
    
    # scale up the dimensionality of the filter bank before the addition to match the depth of the input (1*1 Conv)
    up = Conv2d_bn(mixed, K.int_shape(x)[3], 1, activation=None)
    
    # residual connection
    x = Lambda(lambda inputs, scale: inputs[0] + inputs[1]*scale,
               output_shape=K.int_shape(x)[1:], arguments={'scale': scale},)([x, up])
    
    if activation is not None:
        x = Activation(activation)(x)
    
    return x

In [4]:
img_input = Input(shape=(224, 224, 32))
x = InceptionResNet_block(img_input, 0.1, 'IncRes_A')
print(x)

x = InceptionResNet_block(img_input, 0.1, 'IncRes_B')
print(x)

x = InceptionResNet_block(img_input, 0.1, 'IncRes_C')
print(x)

Tensor("activation_6/Identity:0", shape=(None, 224, 224, 32), dtype=float32)
Tensor("activation_11/Identity:0", shape=(None, 224, 224, 32), dtype=float32)
Tensor("activation_16/Identity:0", shape=(None, 224, 224, 32), dtype=float32)


## (c) Revise the VGG16 Model with Residual Inception Modules

- Conv2D in Block 3 -> `InceptionV1_block` -> `IncRes_A`
- Conv2D in Block 5 -> `InceptionV3_block` -> `IncRes_B`

In [5]:
def VGG16_IncRes(include_top=True, input_tensor=None, input_shape=(224,224,1), pooling='max', classes=1000):
    
    img_input = Input(shape=input_shape)
    
    # Black 1
    x = Conv2d_bn(img_input, 64, (3, 3), activation='relu', padding='same', name='block1_conv1')
    x = Conv2d_bn(x, 64, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2d_bn(x, 128, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = Conv2d_bn(x, 128, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = InceptionResNet_block(x, 0.1, 'IncRes_A')
    x = InceptionResNet_block(x, 0.1, 'IncRes_A')
    x = InceptionResNet_block(x, 0.1, 'IncRes_A')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
    # Block 4
    x = Conv2d_bn(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = Conv2d_bn(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = Conv2d_bn(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    
    # Block 5
    x = InceptionResNet_block(x, 0.1, 'IncRes_B')
    x = InceptionResNet_block(x, 0.1, 'IncRes_B')
    x = InceptionResNet_block(x, 0.1, 'IncRes_B')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    if include_top:
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
    
    inputs = img_input
    model = Model(inputs, x, name='vgg16_inception')
    
    return model

In [6]:
model = VGG16_IncRes(include_top=True)
model.summary()

Model: "vgg16_inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
block1_conv1_conv (Conv2D)      (None, 224, 224, 64) 576         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 224, 224, 64) 192         block1_conv1_conv[0][0]          
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 224, 224, 64) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________